In [3]:
import sys
for p in ['..']:
  if p not in sys.path:
    sys.path.insert(0, p)

from collections import namedtuple
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import scipy.io as spio
from sklearn.linear_model import LogisticRegression

from digits.data import Env
from digits.explore import explore

%matplotlib inline

env = Env('..')
env.assert_ready()
print("ready")

ready
